這份 Notebook 進行 RAG 評估，包括合成評估資料


if colab

In [11]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [12]:
# !pip install openai

In [13]:
# !pip install braintrust autoevals

In [1]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')
braintrust_api_key = os.getenv('BRAINTRUST_API_KEY')


In [15]:
from braintrust import init_logger, traced, wrap_openai, Eval
from openai import OpenAI

logger = init_logger(project="Course-202504", api_key=braintrust_api_key)
client = wrap_openai(OpenAI(api_key=openai_api_key))

if colab 

In [16]:
# braintrust_api_key = userdata.get('braintrust_api_key')

# import os
# os.environ['OPENAI_API_KEY'] = openai_api_key

# from braintrust import init_logger, traced, wrap_openai, Eval
# from openai import OpenAI

# logger = init_logger(project="Course-202504", api_key=braintrust_api_key)
# client = wrap_openai(OpenAI(api_key=openai_api_key))



## 評估資料準備: 合成資料 Synthetic Data

若要人工製作 dataset，雖然品質好，但是實在太太辛苦了，可以怎麼辦?

可用合成 dataset 策略! 用 LLM 幫我們產生評估資料集

我們用合成的，原理是:

1. 針對要做 RAG 的文本先拆 chunks，也就是 contexts
2. 針對 context 用 LLM 產生對應的 "問題" 和 "參考答案"

more: https://ihower.tw/notes/%E6%8A%80%E8%A1%93%E7%AD%86%E8%A8%98-AI/Synthetic+Data+%E5%90%88%E6%88%90%E8%B3%87%E6%96%99

In [3]:
import urllib.request

url = "https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/114/1140224.pdf"
output_path = "1140224.pdf"

try:
    print(f"正在下載檔案: {url}")
    urllib.request.urlretrieve(url, output_path)
    print(f"下載完成，已儲存為 {output_path}")
except Exception as e:
    print(f"下載失敗: {e}")

# !wget https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/114/1140224.pdf

正在下載檔案: https://www.megabank.com.tw/-/media/mega/files/bank/personal/fund/bulletin/weekly-journal/market-analysis/114/1140224.pdf
下載完成，已儲存為 1140224.pdf


In [18]:
!pip install pymupdf

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/



[notice] A new release of pip is available: 24.1.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import fitz # PyMuPDF library
pages = fitz.open("1140224.pdf")

In [16]:
from typing import List
from pydantic import Field, BaseModel

class QAPair(BaseModel):
    reference: str = Field(..., description="The exact text segment from the original context that this Q&A is based on")
    question: str = Field(description="A single question about the content")
    answer: str = Field(..., description="Answer")

class QAPairs(BaseModel):
    pairs: List[QAPair] = Field(..., description="List of question/answer pairs")

@traced
def produce_questions(content):
    completion = client.beta.chat.completions.parse(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "user",
                "content": f"""Please generate 2 question/answer pairs from the following text, focusing specifically on investment and personal finance topics.
For each pair, provide a single question, a unique answer, and include the exact text segment from the original context that the Q&A is based on.

IMPORTANT:
1. Focus ONLY on investment, financial planning, wealth management, stock market, retirement planning, tax optimization, or other personal finance related topics.
2. All questions and answers MUST be in Traditional Chinese (Taiwan).
3. Use terminology and expressions commonly used in Taiwan's financial sector.
4. If the context doesn't contain finance-related information, extract the most relevant aspects that could be applied to personal finance or investment decisions.
5. For each Q&A pair, include the exact text from the original context that contains the information used for the Q&A. This should be copied verbatim from the input context.

Context: <context>{content}</context>""",
            }
        ],
        response_format=QAPairs
    )

    parsed_result = completion.choices[0].message.parsed
    pairs = parsed_result.pairs
    return pairs

In [18]:
x = produce_questions('兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現')

h = x[0].model_dump()

In [19]:
h['test'] = 1234
h

{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷',
 'question': '兆豐銀行投資意見的依據為何？',
 'answer': '兆豐銀行提供的投資意見是根據當時資料情況進行分析和判斷的。',
 'test': 1234}

In [20]:
dataset = []
for idx, page in enumerate(pages):
    context = page.get_text()
    pairs = produce_questions(context)
    for pair in pairs:
      h = pair.model_dump()
      h["page_index"] = idx
      print(h)
      dataset.append(h)

{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現', 'question': '兆豐銀行提供的投資意見是否保證未來績效？', 'answer': '兆豐銀行的投資意見是基於當時資料與情況進行判斷，但因市場變化，投資標的價格與收益會隨時變動，故不保證未來績效表現。', 'page_index': 0}
{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現', 'question': '投資標的的價格與收益會受到什麼因素影響？', 'answer': '投資標的的價格與收益會因市場變化而隨時變動，因此投資者需注意市場風險可能帶來的影響。', 'page_index': 0}
{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現', 'question': '兆豐銀行在提供投資意見時，如何因應市場變化？', 'answer': '兆豐銀行的投資意見是根據當時情況與資料進行判斷，但因市場變化而可能調整，且投資標的的價格與收益會隨時變動，不保證未來績效。', 'page_index': 1}
{'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現', 'question': '投資人應如何理解兆豐銀行的投資建議？', 'answer': '投資人應了解兆豐銀行的投資建議是基於當時資料製作，市場價格與收益會隨時變動，因此投資結果並不保證未來績效，需謹慎評估。', 'page_index': 1}
{'reference': '3 市 場 焦 點 聚 焦 議 題 資 產 觀 點 市 場 回 顧 資料來源：Bloomberg，2025/2/21 全球主要股票指數(%) 過去1個月 漲跌幅% 過去1年 漲跌幅% 年初迄今 漲跌幅% MSCI AC世界指數 1.20% 16

In [21]:
eval_dataset = []
for qa in dataset:
    eval_dataset.append(
        {
            "input": qa['question'],
            "expected": qa['answer'],
            "metadata": {
                "reference": qa['reference'],
                'page_index': qa['page_index'],
                'file_name': '1140224.pdf'
            },
        }
    )

## Baseline (No RAG)

In [22]:
def simple_qa(question):
    completion = client.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {
                "role": "user",
                "content": question,
            }
        ],
    )
    return completion.choices[0].message.content

In [23]:
eval_dataset

[{'input': '兆豐銀行提供的投資意見是否保證未來績效？',
  'expected': '兆豐銀行的投資意見是基於當時資料與情況進行判斷，但因市場變化，投資標的價格與收益會隨時變動，故不保證未來績效表現。',
  'metadata': {'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現',
   'page_index': 0,
   'file_name': '1140224.pdf'}},
 {'input': '投資標的的價格與收益會受到什麼因素影響？',
  'expected': '投資標的的價格與收益會因市場變化而隨時變動，因此投資者需注意市場風險可能帶來的影響。',
  'metadata': {'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現',
   'page_index': 0,
   'file_name': '1140224.pdf'}},
 {'input': '兆豐銀行在提供投資意見時，如何因應市場變化？',
  'expected': '兆豐銀行的投資意見是根據當時情況與資料進行判斷，但因市場變化而可能調整，且投資標的的價格與收益會隨時變動，不保證未來績效。',
  'metadata': {'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與收益將隨時變動，亦不必然為未來績效表現',
   'page_index': 1,
   'file_name': '1140224.pdf'}},
 {'input': '投資人應如何理解兆豐銀行的投資建議？',
  'expected': '投資人應了解兆豐銀行的投資建議是基於當時資料製作，市場價格與收益會隨時變動，因此投資結果並不保證未來績效，需謹慎評估。',
  'metadata': {'reference': '兆豐銀行所做任何投資意見與市場分析結果，係依據資料製作當時情況進行判斷\n惟可能因市場變化而變動，投資標的之價格與

In [24]:
import autoevals

Eval(
    name="Course-202504",
    experiment_name="No RAG",
    data=eval_dataset,
    task=simple_qa,
    scores=[autoevals.Factuality(model="gpt-4.1")],
)

Experiment No RAG is running at https://www.braintrust.dev/app/ispan/p/Course-202504/experiments/No%20RAG
`Eval()` was called from an async context. For better performance, it is recommended to use `await EvalAsync()` instead.


<Task pending name='Task-3' coro=<_EvalCommon.<locals>.run_to_completion() running at c:\Users\xdxd2\AppData\Local\Programs\Python\Python311\Lib\site-packages\braintrust\framework.py:688>>

Course-202504 [experiment_name=No RAG] (data): 32it [00:00, 31994.69it/s]


Course-202504 [experiment_name=No RAG] (tasks):   0%|          | 0/32 [00:00<?, ?it/s]

c:\Users\xdxd2\AppData\Local\Programs\Python\Python311\Lib\functools.py:76: RuntimeWarning: coroutine 'produce_questions_main' was never awaited
  return partial(update_wrapper, wrapped=wrapped,
c:\Users\xdxd2\AppData\Local\Programs\Python\Python311\Lib\functools.py:76: RuntimeWarning: coroutine 'AsyncBetaCompletionsV1Wrapper.parse' was never awaited
  return partial(update_wrapper, wrapped=wrapped,



=========================SUMMARY=========================
48.75% 'Factuality' score

1751278937.17s start
1751278952.14s end
8.61s duration
8.54s llm_duration
33.06tok prompt_tokens
466.03tok completion_tokens
499.09tok total_tokens
0.00$ estimated_cost
0tok prompt_cached_tokens
0tok prompt_cache_creation_tokens

See results for No RAG at https://www.braintrust.dev/app/ispan/p/Course-202504/experiments/No%20RAG


## Naive RAG

我們用 braintrust 提供的 autoevals 套件中的內建評估 prompt: Factuality  https://github.com/braintrustdata/autoevals/blob/main/templates/factuality.yaml

這讓 AI 拿你 task 的 output 和 dataset 中的 expected，比較兩個回答之間的事實內容差異，常見於自動評估問答或教學模型輸出的正確性。以下是詳細解釋：

* (A)	提交的回答是專家答案的子集合，且完全一致。（例：專家說 A+B+C，提交只說 A+C，但沒說錯）
* (B)	提交的回答是專家答案的超集合，也就是它包含更多細節，但與專家內容一致。（例：專家說 A+C，提交說 A+B+C+D）
* (C)	提交的回答與專家答案完全一樣，所有細節一致
* (D)	兩者有事實上的衝突或矛盾
* (E)	雖然兩者不完全一樣，但差異不影響事實正確性（可能是換句話說、順序不同等）


* C 與 E 是「最理想」的狀態，得分是 1
* B 比 A 更完整，分數略高。0.6 跟 0.4
* D 是錯誤，得 0 分。




In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 103.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.2 MB/s eta 0:00:00

In [ ]:
import chromadb
chroma_client = chromadb.Client()

collection = chroma_client.create_collection(name="collection3")

In [ ]:
!pip install tiktoken

In [ ]:
!pip install langchain_text_splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import tiktoken

tokenizer = tiktoken.get_encoding("o200k_base") # gpt-4o 是 o200k_base，之前版本 gpt-4-turbo 和 gpt-3.5-turbo 是 cl100k_base

def length_function(text: str):
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(length_function=length_function, chunk_size=800, chunk_overlap=200, separators=[
    "\n\n",
    "\n",
    " ",
    ".",
    ",",
    "\u200b",  # Zero-width space
    "\uff0c",  # Fullwidth comma ，
    "\u3001",  # Ideographic comma 、
    "\uff0e",  # Fullwidth full stop ．
    "\u3002",  # Ideographic full stop 。
    "",
])

In [ ]:
def get_embeddings(text):
  response = client.embeddings.create(
      input=text,
      model="text-embedding-3-small"
  )

  return response.data[0].embedding

In [ ]:
for idx,page in enumerate(pages):
  chunks = text_splitter.split_text(page.get_text())

  collection.add(
    documents = chunks,
    embeddings = [ get_embeddings(chunk) for chunk in chunks ],
    ids=[f"doc-1-page-{idx}-chunk-{x}" for x in range( len(chunks) ) ]
  )

In [ ]:
from typing import List
from pydantic import Field, BaseModel

class QueryResult(BaseModel):
    relevant_quotes: List[str]
    answer: str
    following_questions: List[str]

@traced
def ask_with_rag(question):
  results = collection.query(
      query_embeddings = get_embeddings(question),
      # 可有 where 參數可針對上述的 metadatas 做過濾，例如日期、頁數等
      n_results=10
  )

  documents = results['documents'][0]
  context = '\n'.join('* ' + doc for doc in documents)

  user_prompt = f"""
  I will provide you with a document and then ask you a question about it. Please respond following these steps:

  <document>
  {context}
  </document>

  Question: {question}

  Please answer in the following format:

  1. First, identify the most relevant quotes from the document that help answer the question and list them. Each quote should be relatively short.
    If there are no relevant quotes, write "No relevant quotes".

  2. Then, answer the question using facts from these quotes without directly referencing the content in your answer.

  3. Finally, provide 3 related follow-up questions based on the original question and document content that would help explore the topic further.

  If the document does not contain sufficient information to answer the question, please state this in the answer field, but still provide any relevant quotes (if available) and possible follow-up questions.
  Please respond in Traditional Chinese (Taiwan).
  """

  completion = client.beta.chat.completions.parse(
      model="gpt-4.1-mini",
      messages=[
          {"role": "user", "content": user_prompt},
      ],
      response_format=QueryResult
  )

  parsed_result = completion.choices[0].message.parsed
  return parsed_result.answer

In [ ]:
import autoevals

Eval(
    name="Course-202504",
    experiment_name="Naive RAG",
    data=eval_dataset,
    task=ask_with_rag,
    scores=[autoevals.Factuality(model="gpt-4.1")],
)

Experiment Naive RAG-addfce82 is running at https://www.braintrust.dev/app/ihower/p/Course-202504/experiments/Naive%20RAG-addfce82


<Task pending name='Task-657' coro=<_EvalCommon.<locals>.run_to_completion() running at /usr/local/lib/python3.11/dist-packages/braintrust/framework.py:656>>

## 使用 Ragas 指標來做評估

https://docs.ragas.io/en/stable/concepts/metrics/available_metrics/

參考自 https://www.braintrust.dev/docs/cookbook/recipes/SimpleRagas

In [ ]:
@traced
def fetch_top_k_relevant_sections(question):
  results = collection.query(
      query_embeddings = get_embeddings(question),
      # 可有 where 參數可針對上述的 metadatas 做過濾，例如日期、頁數等
      n_results=10
  )

  documents = results['documents'][0]
  return documents

@traced
def generate_answer_from_docs(question, retrieved_content):
  context = '\n'.join('* ' + doc for doc in retrieved_content)

  user_prompt = f"""
  I will provide you with a document and then ask you a question about it. Please respond following these steps:

  <document>
  {context}
  </document>

  Question: {question}

  Please answer in the following format:

  1. First, identify the most relevant quotes from the document that help answer the question and list them. Each quote should be relatively short.
    If there are no relevant quotes, write "No relevant quotes".

  2. Then, answer the question using facts from these quotes without directly referencing the content in your answer.

  3. Finally, provide 3 related follow-up questions based on the original question and document content that would help explore the topic further.

  If the document does not contain sufficient information to answer the question, please state this in the answer field, but still provide any relevant quotes (if available) and possible follow-up questions.
  Please respond in Traditional Chinese (Taiwan).
  """

  completion = client.beta.chat.completions.parse(
      model="gpt-4.1-mini",
      messages=[
          {"role": "user", "content": user_prompt},
      ],
      response_format=QueryResult
  )

  parsed_result = completion.choices[0].message.parsed
  return parsed_result

@traced
def generate_answer_e2e(question):
  retrieved_content = fetch_top_k_relevant_sections(question)
  result = generate_answer_from_docs(question, retrieved_content)

  return { "answer": result.answer, "retrieved_docs": retrieved_content }

In [ ]:
from braintrust import EvalAsync

from autoevals import AnswerCorrectness, ContextRecall, ContextPrecision, Faithfulness

# Wrap ContextRecall() to propagate along the "answer" and "context" values separately
async def context_recall(output, **kwargs):
    return await ContextRecall(model="gpt-4.1").eval_async(
        output=output["answer"], context=output["retrieved_docs"], **kwargs
    )

async def context_precision(output, **kwargs):
    return await ContextPrecision(model="gpt-4.1").eval_async(
        output=output["answer"], context=output["retrieved_docs"], **kwargs
    )

async def faithfulness(output, **kwargs):
    return await Faithfulness(model="gpt-4.1").eval_async(
        output=output["answer"], context=output["retrieved_docs"], **kwargs
    )

async def answer_correctness(output, **kwargs):
    return await AnswerCorrectness(model="gpt-4.1").eval_async(output=output["answer"], **kwargs)

eval_result = await EvalAsync(
    name="Course-202504",
    experiment_name="Ragas",
    data=eval_dataset,
    task=generate_answer_e2e,
    scores=[context_recall, context_precision, faithfulness, answer_correctness],
    metadata=dict(model='gpt-4.1-mini', top_k=10),
)

Experiment Ragas is running at https://www.braintrust.dev/app/ihower/p/Course-202504/experiments/Ragas
Course-202504 [experiment_name=Ragas] (data): 32it [00:00, 24092.21it/s]


Course-202504 [experiment_name=Ragas] (tasks):   0%|          | 0/32 [00:00<?, ?it/s]

Found exceptions for the following scorers: faithfulness [KeyError('verdict')]
Found exceptions for the following scorers: faithfulness [KeyError('verdict')]
Found exceptions for the following scorers: context_recall [TypeError("unsupported operand type(s) for +: 'int' and 'NoneType'")]
Found exceptions for the following scorers: faithfulness [KeyError('verdict')]
Found exceptions for the following scorers: faithfulness [KeyError('verdict')]



=========================SUMMARY=========================
Ragas compared to Naive RAG-addfce82:
59.56% 'AnswerCorrectness' score
100.00% 'ContextPrecision'  score
97.31% 'ContextRecall'     score
84.43% 'Faithfulness'      score

1745942477.20s start
1745942541.03s end
55.08s (-12293.44%) 'duration'         	(32 improvements, 0 regressions)
3.08s (-52.22%) 'llm_duration'     	(23 improvements, 9 regressions)
5646.81tok (-) 'prompt_tokens'    	(0 improvements, 0 regressions)
407.78tok (+1003.12%) 'completion_tokens'	(15 improvements, 16 regressions)
6054.59tok (+1003.12%) 'total_tokens'     	(15 improvements, 16 regressions)
0.00$ (+00.00%) 'estimated_cost'   	(8 improvements, 10 regressions)

See results for Ragas at https://www.braintrust.dev/app/ihower/p/Course-202504/experiments/Ragas
